# Yandex.Afisha marketing analysis

## Contents <a id='back'></a>

1. [Introduction](#introduction)
2. [Data Loading and Inspection](#data_loading_and_inspection)
3. [Reports and Metrics](#reports_and_metrics)


## 1. Introduction <a id='introduction'></a>

This is the project for the "Business Analytics" of the Tripleten's DA course. 

We have been tasked with optimizing marketing spending for Yandex.Afisha. To accomplish this, we have access to visits logs, orders and marketing expenses.

[Back to Contents](#back)

## 2. Data Loading and Inspection <a id='data-loading-and-inspection'></a>

For this project we'll be using the following:
- Python 3.9.5
- Pandas 1.2.4
- Seaborn 0.11.1
- Matplotlib.pyplot 3.3.4
- Plotly express 5.4.0

Versions were chosen so they match as closely as possible the versions available on the Tripleten servers

In [21]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px

There are three data files to read:
- visits_log_us.csv: server logs with data on website visits
- orders_log_us.csv: data on orders
- costs_us.csv: data on marketing expenses

In [2]:
def load_csv(file: str):
    try: # Local file path
        filepath=f'datasets/{file}'
        df = pd.read_csv(filepath)
    except FileNotFoundError: # Tripleten server file path
        filepath=f'/datasets/{file}'
        df = pd.read_csv(filepath)
    return df

In [3]:
visits = load_csv('visits_log_us.csv')
orders = load_csv('orders_log_us.csv')
costs = load_csv('costs_us.csv')

The next step is to do the following to each table: 

- check for missing data
- check for duplicates
- correct column names
- correct data types

Lets create some functions to help with that.

In [4]:
def full_info(df: pd.DataFrame):
    df.info(memory_usage='deep')
    display(df.head(5))
    print(f'Duplicated rows: {df.duplicated().sum()}')

In [5]:
def format_columns(names: list):
    fixed_names = []
    for name in names:
        fixed = name.lower()
        fixed = fixed.replace(' ', '_')
        fixed_names.append(fixed)
    return fixed_names

### Visits data

In [6]:
full_info(visits)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 359400 entries, 0 to 359399
Data columns (total 5 columns):
 #   Column     Non-Null Count   Dtype 
---  ------     --------------   ----- 
 0   Device     359400 non-null  object
 1   End Ts     359400 non-null  object
 2   Source Id  359400 non-null  int64 
 3   Start Ts   359400 non-null  object
 4   Uid        359400 non-null  uint64
dtypes: int64(1), object(3), uint64(1)
memory usage: 79.3 MB


,Device,End Ts,Source Id,Start Ts,Uid
0,touch,2017-12-20 17:38:00,4,2017-12-20 17:20:00,16879256277535980062
1,desktop,2018-02-19 17:21:00,2,2018-02-19 16:53:00,104060357244891740
2,touch,2017-07-01 01:54:00,5,2017-07-01 01:54:00,7459035603376831527
3,desktop,2018-05-20 11:23:00,9,2018-05-20 10:59:00,16174680259334210214
4,desktop,2017-12-27 14:06:00,3,2017-12-27 14:06:00,9969694820036681168


Duplicated rows: 0


No missing or duplicated values.

In [7]:
visits.columns = format_columns(visits.columns)

In [8]:
date_format = '%Y-%m-%d %H:%M:%S'
visits['start_ts'] = pd.to_datetime(visits['start_ts'], format=date_format)
visits['end_ts'] = pd.to_datetime(visits['end_ts'], format=date_format)

visits['device'] = visits['device'].astype('category')

Let's check that everything is correct now.

In [9]:
full_info(visits)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 359400 entries, 0 to 359399
Data columns (total 5 columns):
 #   Column     Non-Null Count   Dtype         
---  ------     --------------   -----         
 0   device     359400 non-null  category      
 1   end_ts     359400 non-null  datetime64[ns]
 2   source_id  359400 non-null  int64         
 3   start_ts   359400 non-null  datetime64[ns]
 4   uid        359400 non-null  uint64        
dtypes: category(1), datetime64[ns](2), int64(1), uint64(1)
memory usage: 11.3 MB


,device,end_ts,source_id,start_ts,uid
0,touch,2017-12-20 17:38:00,4,2017-12-20 17:20:00,16879256277535980062
1,desktop,2018-02-19 17:21:00,2,2018-02-19 16:53:00,104060357244891740
2,touch,2017-07-01 01:54:00,5,2017-07-01 01:54:00,7459035603376831527
3,desktop,2018-05-20 11:23:00,9,2018-05-20 10:59:00,16174680259334210214
4,desktop,2017-12-27 14:06:00,3,2017-12-27 14:06:00,9969694820036681168


Duplicated rows: 0


### Orders data

In [10]:
full_info(orders)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50415 entries, 0 to 50414
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   Buy Ts   50415 non-null  object 
 1   Revenue  50415 non-null  float64
 2   Uid      50415 non-null  uint64 
dtypes: float64(1), object(1), uint64(1)
memory usage: 4.4 MB


,Buy Ts,Revenue,Uid
0,2017-06-01 00:10:00,17.00,10329302124590727494
1,2017-06-01 00:25:00,0.55,11627257723692907447
2,2017-06-01 00:27:00,0.37,17903680561304213844
3,2017-06-01 00:29:00,0.55,16109239769442553005
4,2017-06-01 07:58:00,0.37,14200605875248379450


Duplicated rows: 0


No missing or duplicated values.

In [11]:
orders.columns = format_columns(orders.columns)

In [12]:
date_format = '%Y-%m-%d %H:%M:%S'
orders['buy_ts'] = pd.to_datetime(orders['buy_ts'], format=date_format)

Final check to see that everything is ok.

In [13]:
full_info(orders)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50415 entries, 0 to 50414
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype         
---  ------   --------------  -----         
 0   buy_ts   50415 non-null  datetime64[ns]
 1   revenue  50415 non-null  float64       
 2   uid      50415 non-null  uint64        
dtypes: datetime64[ns](1), float64(1), uint64(1)
memory usage: 1.2 MB


,buy_ts,revenue,uid
0,2017-06-01 00:10:00,17.00,10329302124590727494
1,2017-06-01 00:25:00,0.55,11627257723692907447
2,2017-06-01 00:27:00,0.37,17903680561304213844
3,2017-06-01 00:29:00,0.55,16109239769442553005
4,2017-06-01 07:58:00,0.37,14200605875248379450


Duplicated rows: 0


### Costs data

In [14]:
full_info(costs)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2542 entries, 0 to 2541
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   source_id  2542 non-null   int64  
 1   dt         2542 non-null   object 
 2   costs      2542 non-null   float64
dtypes: float64(1), int64(1), object(1)
memory usage: 206.2 KB


,source_id,dt,costs
0,1,2017-06-01,75.20
1,1,2017-06-02,62.25
2,1,2017-06-03,36.53
3,1,2017-06-04,55.00
4,1,2017-06-05,57.08


Duplicated rows: 0


Again, no missing or duplicated values. Column names are correctly formated.

In [15]:
date_format = '%Y-%m-%d'
costs['dt'] = pd.to_datetime(costs['dt'], format=date_format)

Final check.

In [16]:
full_info(costs)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2542 entries, 0 to 2541
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   source_id  2542 non-null   int64         
 1   dt         2542 non-null   datetime64[ns]
 2   costs      2542 non-null   float64       
dtypes: datetime64[ns](1), float64(1), int64(1)
memory usage: 59.7 KB


,source_id,dt,costs
0,1,2017-06-01,75.20
1,1,2017-06-02,62.25
2,1,2017-06-03,36.53
3,1,2017-06-04,55.00
4,1,2017-06-05,57.08


Duplicated rows: 0


The dataset is ready. Let's get working.

[Back to Contents](#back)

## Reports and Metrics<a id="reports_and_metrics"></a>

### Product 
Let's see how users interact with the product. We'll visualize DAU, WAU and MAU, as well as sessions per day, session length and user retention rate.

For that, we have to extract the date, week and month of each visit.

In [44]:
visits['year'] = visits['start_ts'].dt.isocalendar().year
visits['month'] = visits['start_ts'].astype('datetime64[M]')
visits['week'] = visits['start_ts'].astype('datetime64[W]')
visits['date'] = visits['start_ts'].dt.date

In [45]:
visits.head()

,device,end_ts,source_id,start_ts,uid,year,month,week,date
0,touch,2017-12-20 17:38:00,4,2017-12-20 17:20:00,16879256277535980062,2017,2017-12-01,2017-12-14,2017-12-20
1,desktop,2018-02-19 17:21:00,2,2018-02-19 16:53:00,104060357244891740,2018,2018-02-01,2018-02-15,2018-02-19
2,touch,2017-07-01 01:54:00,5,2017-07-01 01:54:00,7459035603376831527,2017,2017-07-01,2017-06-29,2017-07-01
3,desktop,2018-05-20 11:23:00,9,2018-05-20 10:59:00,16174680259334210214,2018,2018-05-01,2018-05-17,2018-05-20
4,desktop,2017-12-27 14:06:00,3,2017-12-27 14:06:00,9969694820036681168,2017,2017-12-01,2017-12-21,2017-12-27


**Daily, Weekly and Monthly Active Users**

Let's visualize the evolution of active users over the year. The calculation of these three metrics is quite similar, so we can create a function to do it.

In [55]:
# Calculates DAU, WAU or MAU. The period needs to match the column name of the visits dataframe
def AU(period: str):
    new_au = visits.groupby(period).agg({'uid': ['nunique', 'count']}).reset_index()
    new_au.columns = [period, 'n_users', 'n_sessions']
    new_au['sessions_per_user'] = new_au['n_sessions'] / new_au['n_users']
    return new_au

**DAU**

In [66]:
# Calculate DAU
dau = AU('date')

# Plot the line chart
fig = px.line(dau,
              x='date',
              y='n_users',
              title='DAU',
              labels={'date': 'Date',
                      'n_users': 'Users'}
              )

# Add the 'average' line
dau_total = dau['n_users'].mean()
fig.add_hline(y=dau_total,
              line_color='green',
              line_dash='dash',
              annotation_text=f'Average DAU: {dau_total: .0f}',
              annotation_position='top left')

fig.show()

**WAU**

In [54]:
# Calculate WAU
wau = AU('week')

# Plot the line chart
fig=px.line(wau,
            title='WAU',
            x='week',
            y='n_users',
            labels={'week': 'Week',
                    'n_users': 'Users'
                    }            
            )

# Add the 'average' line
wau_total = wau['n_users'].mean()
fig.add_hline(y=wau_total,
              line_color='green',
              line_dash='dash',
              annotation_text=f'Average WAU: {wau_total: .0f}',
              annotation_position='top left'
              )

fig.show()

**Note:** The drop on the last week is because the dataset ends on the first day of that week.

**MAU**

In [65]:
# Calculate MAU
mau = AU('month')

# Plot the line chart
fig = px.line(mau,
              title='MAU',
              x='month',
              y='n_users',
              labels={'month': 'Month',
                      'n_users': 'Users'}
              )

# Add the 'average' line
mau_total = mau['n_users'].mean()
fig.add_hline(y=mau_total,
              line_color='green',
              line_dash='dash',
              annotation_text=f'Average MAU: {mau_total: .0f}',
              annotation_position='top left'
              )

fig.show()

There is an increase of activity starting around Sep 21, and lasts until Mar 22.

On Mar 30 and 31, there was an unusual activity drop. Having a single visitor in the whole day suggest that the service was unavailable during that time.

WAU and DAU charts show that there has been an increase on users during this year.

**Sessions per day**

We already calculated sessions per day in the last step.

In [72]:
fig = px.line(dau,
              title='Daily sessions',
              x='date',
              y=['n_sessions', 'n_users'],
              labels={'n_sessions': 'Sessions',
                      'n_users': 'Users'})

fig.show()

As expected, the daily sessions track closely with daily users.

In [ ]:
fig = px.line(dau,
             x='date',
             y='sessions_per_user',
             title='Sessions per User',
             labels={'sessions_per_user': 'sessions'})

fig.show()

The daily sessions hover slightly above 1. How abouth monthly sessions?

In [74]:
fig = px.line(mau,
             x='month',
             y='sessions_per_user',
             title='Monthly Sessions per User',
             labels={'sessions_per_user': 'sessions'})

fig.show()

The monthly sessions are between 1.24 and 1.34. 